In [48]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

pd.set_option("display.max_rows", None, "display.max_columns", None)

# Import our input dataset
charity_df = pd.read_csv('Resources/charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [49]:
# Name and EIN are unique identifiers and will not aid in machine learning, drop them
charity_ml_df  = charity_df.drop("EIN",1)
charity_ml_df  = charity_ml_df.drop("NAME",1)
charity_ml_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [50]:
# Generate our categorical variable list
charity_cat = charity_ml_df.dtypes[charity_ml_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [51]:
# check application type value counts
application_type_counts = charity_ml_df.APPLICATION_TYPE.value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [52]:
# Group all applications with less than 1000 into other category.
# Determine which values to replace
replace_ap_type = list(application_type_counts[application_type_counts < 1000].index)

# Replace in DataFrame
for ap_type in replace_ap_type:
    charity_ml_df.APPLICATION_TYPE = charity_ml_df.APPLICATION_TYPE.replace(ap_type,"Other")


# Check to make sure binning was successful
charity_ml_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [53]:
# check classification value counts
classification_counts = charity_ml_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C2400        6
C4100        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C3200        2
C1256        2
C1234        2
C1267        2
C1246        2
C4200        1
C1283        1
C3700        1
C1732        1
C4500        1
C2600        1
C1580        1
C2190        1
C5200        1
C1248        1
C1245        1
C1728        1
C1370        1
C2500        1
C2561        1
C2150        1
C2380        1
C4120        1
C1900        1
C1820        1
C8210        1
C6100     

In [54]:
# with goal to get <10 variables, will group all applications with less than 500 into other category.
# Determine which values to replace
replace_classification = list(classification_counts[classification_counts < 500].index)

# Replace in DataFrame
for class_type in replace_classification:
    charity_ml_df.CLASSIFICATION = charity_ml_df.CLASSIFICATION.replace(class_type,"Other")


# Check to make sure binning was successful
charity_ml_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [55]:
# Check the number of unique values in each column
charity_ml_df[charity_cat].nunique()

APPLICATION_TYPE          6
AFFILIATION               6
CLASSIFICATION            7
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [56]:
# check income value counts
income_counts = charity_ml_df.INCOME_AMT.value_counts()
income_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [57]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_ml_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [58]:
# Merge one-hot encoded features and drop the originals
charity_ml_df = charity_ml_df.merge(encode_df,left_index=True, right_index=True)
charity_ml_df = charity_ml_df.drop(charity_cat,1)
charity_ml_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [59]:
# Split our preprocessed data into our features and target arrays
y = charity_ml_df["IS_SUCCESSFUL"].values
X = charity_ml_df.drop(["IS_SUCCESSFUL", "SPECIAL_CONSIDERATIONS_N"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [60]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [61]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 3)                 123       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 4         
Total params: 127
Trainable params: 127
Non-trainable params: 0
_________________________________________________________________


In [62]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 82us/sample - loss: 0.6408 - accuracy: 0.6608
Epoch 2/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5830 - accuracy: 0.7156
Epoch 3/100
25724/25724 [==============================] - 1s 46us/sample - loss: 0.5734 - accuracy: 0.7196
Epoch 4/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5686 - accuracy: 0.7229
Epoch 5/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5658 - accuracy: 0.7269
Epoch 6/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5646 - accuracy: 0.7268
Epoch 7/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5638 - accuracy: 0.7273
Epoch 8/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5633 - accuracy: 0.7271
Epoch 9/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5629 - accuracy: 0.7271
Epoch

25724/25724 [==============================] - 1s 48us/sample - loss: 0.5591 - accuracy: 0.7275
Epoch 76/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5592 - accuracy: 0.7277
Epoch 77/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5590 - accuracy: 0.7267
Epoch 78/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5591 - accuracy: 0.7275
Epoch 79/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5592 - accuracy: 0.7276
Epoch 80/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5590 - accuracy: 0.7277
Epoch 81/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5591 - accuracy: 0.7282
Epoch 82/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5591 - accuracy: 0.7276
Epoch 83/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5591 - accuracy: 0.7283
Epoch 84/100
25724/25724 [======

In [63]:
# Optimize model attempt 1
# add additional hidden layer

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  3
hidden_nodes_layer2 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 3)                 123       
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 3         
Total params: 134
Trainable params: 134
Non-trainable params: 0
_________________________________________________________________


In [64]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 80us/sample - loss: 0.6502 - accuracy: 0.6843
Epoch 2/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5949 - accuracy: 0.7220
Epoch 3/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5822 - accuracy: 0.7240
Epoch 4/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5780 - accuracy: 0.7250
Epoch 5/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5767 - accuracy: 0.7266
Epoch 6/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5757 - accuracy: 0.7261
Epoch 7/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5753 - accuracy: 0.7259s - loss: 0.577
Epoch 8/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5743 - accuracy: 0.7259
Epoch 9/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5727 - accurac

25724/25724 [==============================] - 1s 51us/sample - loss: 0.5670 - accuracy: 0.7292
Epoch 76/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5669 - accuracy: 0.7280
Epoch 77/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5670 - accuracy: 0.7289
Epoch 78/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5669 - accuracy: 0.7287
Epoch 79/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5670 - accuracy: 0.7287
Epoch 80/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5670 - accuracy: 0.7287
Epoch 81/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5669 - accuracy: 0.7286
Epoch 82/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5670 - accuracy: 0.7282
Epoch 83/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5669 - accuracy: 0.7282
Epoch 84/100
25724/25724 [======

In [65]:
# Optimize model attempt 2
# add additional neurons in each layer

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 8)                 328       
_________________________________________________________________
dense_22 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 6         
Total params: 379
Trainable params: 379
Non-trainable params: 0
_________________________________________________________________


In [66]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 80us/sample - loss: 0.6199 - accuracy: 0.6632
Epoch 2/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5757 - accuracy: 0.6996
Epoch 3/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5667 - accuracy: 0.7145
Epoch 4/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5629 - accuracy: 0.7172
Epoch 5/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5604 - accuracy: 0.7226
Epoch 6/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5566 - accuracy: 0.7283
Epoch 7/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5554 - accuracy: 0.7291
Epoch 8/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5547 - accuracy: 0.7289
Epoch 9/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5542 - accuracy: 0.7298
Epoch

Epoch 76/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.5485 - accuracy: 0.7317
Epoch 77/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5486 - accuracy: 0.7313
Epoch 78/100
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5486 - accuracy: 0.7317
Epoch 79/100
25724/25724 [==============================] - 2s 63us/sample - loss: 0.5483 - accuracy: 0.7322
Epoch 80/100
25724/25724 [==============================] - 2s 59us/sample - loss: 0.5483 - accuracy: 0.7318
Epoch 81/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5484 - accuracy: 0.7321
Epoch 82/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5484 - accuracy: 0.7313
Epoch 83/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5481 - accuracy: 0.7318
Epoch 84/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.5481 - accuracy: 0.7325
Epoch 85/100
25724/

In [67]:
# optimize model attempt 3
# change activation functions to tanh from relu

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 8)                 328       
_________________________________________________________________
dense_25 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 6         
Total params: 379
Trainable params: 379
Non-trainable params: 0
_________________________________________________________________


In [68]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 89us/sample - loss: 0.5964 - accuracy: 0.6998
Epoch 2/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5667 - accuracy: 0.7223
Epoch 3/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5613 - accuracy: 0.7249
Epoch 4/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5586 - accuracy: 0.7265
Epoch 5/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5571 - accuracy: 0.7269
Epoch 6/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5560 - accuracy: 0.7286
Epoch 7/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5552 - accuracy: 0.7296
Epoch 8/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5545 - accuracy: 0.7296
Epoch 9/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5541 - accuracy: 0.7307
Epoch

25724/25724 [==============================] - 1s 54us/sample - loss: 0.5463 - accuracy: 0.7340
Epoch 76/100
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5459 - accuracy: 0.7342
Epoch 77/100
25724/25724 [==============================] - 2s 74us/sample - loss: 0.5462 - accuracy: 0.7344s - loss: 0.5441 - 
Epoch 78/100
25724/25724 [==============================] - 2s 66us/sample - loss: 0.5461 - accuracy: 0.7348
Epoch 79/100
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5461 - accuracy: 0.7333
Epoch 80/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5457 - accuracy: 0.7343
Epoch 81/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5458 - accuracy: 0.7339
Epoch 82/100
25724/25724 [==============================] - 1s 52us/sample - loss: 0.5459 - accuracy: 0.7341
Epoch 83/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5459 - accuracy: 0.7353
Epoch 84/100


In [72]:
# Optimization attempt 4
# No significant improvement with optimizations, suggesting input data may not be optimized.
# hypothesize that application type and classification do not provide helpful information and may confuse the model
charity_ml_filtered_df = charity_df.drop(["APPLICATION_TYPE", "CLASSIFICATION", "NAME", "EIN"],1)
charity_ml_filtered_df.head()

,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Independent,ProductDev,Association,1,0,N,5000,1
1,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [74]:
# encode remaining categorical variables
charity_cat2 = charity_ml_filtered_df.dtypes[charity_ml_filtered_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_ml_filtered_df[charity_cat2].nunique()

AFFILIATION               6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [75]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df2 = pd.DataFrame(enc.fit_transform(charity_ml_filtered_df[charity_cat2]))

# Add the encoded variable names to the DataFrame
encode_df2.columns = enc.get_feature_names(charity_cat2)
encode_df2.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [76]:
# Merge one-hot encoded features and drop the originals
charity_ml_filtered_df = charity_ml_filtered_df.merge(encode_df2,left_index=True, right_index=True)
charity_ml_filtered_df = charity_ml_filtered_df.drop(charity_cat2,1)
charity_ml_filtered_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [77]:
# Split our preprocessed data into our features and target arrays
y = charity_ml_df["IS_SUCCESSFUL"].values
X = charity_ml_df.drop(["IS_SUCCESSFUL", "SPECIAL_CONSIDERATIONS_N"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [78]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 3)                 123       
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 4         
Total params: 127
Trainable params: 127
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 76us/sample - loss: 0.6420 - accuracy: 0.6721
Epoch 2/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5877 - accuracy: 0.7156
Epoch 3/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5789 - accuracy: 0.7199
Epoch 4/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5744 - accuracy: 0.7242
Epoch 5/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5720 - accuracy: 0.7233
Epoch 6/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5699 - accuracy: 0.7241
Epoch 7/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5688 - accuracy: 0.7239
Epoch 8/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5679 - accuracy: 0.7249
Epoch 9/100
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5673 - accuracy: 0.7247
Epoch

25724/25724 [==============================] - 1s 49us/sample - loss: 0.5606 - accuracy: 0.7251
Epoch 76/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5606 - accuracy: 0.7254
Epoch 77/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5605 - accuracy: 0.7257
Epoch 78/100
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5605 - accuracy: 0.7257
Epoch 79/100
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5605 - accuracy: 0.7257
Epoch 80/100
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5605 - accuracy: 0.7264
Epoch 81/100
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5605 - accuracy: 0.7261
Epoch 82/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5605 - accuracy: 0.7259
Epoch 83/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.5606 - accuracy: 0.7258
Epoch 84/100
25724/25724 [======